In [1]:
def worker(order):
    rows, trace = order
    ans = np.zeros((len(rows), 5))
    for i, r in enumerate(rows):
        mtrace = trace[r[0]: r[1]]
        ans[i, :] = [mtrace.mean(), np.median(mtrace), np.std(mtrace), mtrace.min(), mtrace.max()]
    return ans

def regionprops1d(mask, *_trace, n_proc=1):
    """
    :param mask: 1d numpy array mask of bool, or [0,1]
    :param _trace: traces to process for stats (max, min, std, median, mean)
    :param n_proc: number of processes to spawn while calculating stats for traces
    :return: DataFrame with indices Onset (inclusive), Offset (exclusive), Length (absolute)
             When given _trace, also calculated Min, Max, STD, Median, and Mean for each trace
    """

    assert (len(mask.shape) == 1), 'shape mismatch, mask should be 1d'

    col_names = ['Onset', 'Offset', 'Length']

    if not (mask.any()):
        return None

    diff_mask = np.diff(mask, axis=-1, prepend=0, append=0)
    t_start = np.where(diff_mask == 1)[0]
    t_stop = np.where(diff_mask == -1)[0]
    assert (len(t_start) == len(t_stop))

    res = np.vstack([t_start, t_stop, t_stop - t_start]).T
    df_res = pd.DataFrame(res, columns=col_names, dtype=int)

    for itr, trace in enumerate(_trace):
        assert(len(trace) == len(mask)), f'trace len = {len(trace)} not equal to mask len = {len(mask)}'

        order = [(s, trace) for s in np.array_split(res, n_proc, axis=0)]

        if itr:
            cols = list(map(lambda x: f'{x}_{itr}', ['Mean', 'Median', 'STD', 'Min', 'Max']))
        else:
            cols = ['Mean', 'Median', 'STD', 'Min', 'Max']

        if n_proc > 1:
            with Pool(n_proc) as p:
                ans = pd.DataFrame(np.vstack(p.map(worker, order)), columns=cols)
        else:
            ans = pd.DataFrame(np.vstack(list(map(worker, order))), columns=cols)

        df_res = pd.merge(left=df_res, right=ans, left_index=True, right_index=True)

    return df_res

In [2]:
import pandas as pd
from scipy.io import loadmat
import mne
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import os

In [4]:
files = !ls *.mat
df_data = pd.DataFrame({'mat':files})
df_data

,mat
0,export1_13Nov07.mat
1,export1_15Jan08.mat
2,export1_15Nov07.mat
3,export1_19Dec07.mat
4,export1_20Dec07.mat
5,export1_21Dec07.mat
6,export1_24Jan08.mat
7,export1_2Jan08.mat
8,export1_30Nov07.mat
9,export1_3Jan08.mat


# Single subject

In [5]:
matfile = df_data.mat.loc[0]
print(matfile)
mat = loadmat(matfile)

export1_13Nov07.mat


## Preprocess

In [6]:
raw = mne.io.RawArray(np.concatenate([mat['export_EMG'], 
                mat['export_X'], 
                mat['export_Y'], 
                mat['export_OnPaper'], 
                mat['export_Type'], 
                mat['export_Trial']], axis=-1).T,
               mne.create_info(['emg%i'%i for i in range(8)] + ['X', 'Y', 'OnPaper', 'Type', 'Trial'], 
                               sfreq=1000,
                               ch_types=['emg']*8+['misc']*2+['stim']*3))

raw.apply_hilbert(picks='emg', envelope=True)
raw.filter(0, 5, picks=['emg', 'misc']);
raw.resample(10);

xmin, ymin = raw.get_data(picks=['X', 'Y']).min(-1)
xmax, ymax = raw.get_data(picks=['X', 'Y']).max(-1)

Creating RawArray with float64 data, n_channels=13, n_times=3234809
    Range : 0 ... 3234808 =      0.000 ...  3234.808 secs
Ready.
No data channels found. The highpass and lowpass values in the measurement info will not be updated.
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 5.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Filter length: 1651 samples (1.651 sec)

648 events found
Event IDs: [1]
Trigger channel has a non-zero initial value of 1 (consider using initial_event=True to detect this event)
9 events found
Event IDs: [ 2  3  4  5  6  7  8  9 10]
Trigger channel has a non-zero initial value of 1 (consider using initial_event=True to detect this event)
498 events fo

## OnPaper Decimation

In [7]:
trace = raw.get_data(picks=['OnPaper'])[0]

plt.figure()
plt.hist(trace[trace>0], bins=100)
plt.axvline(0.5, color='r')
plt.yscale('log')
plt.title('Finding OnPaper threshold')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Finding OnPaper threshold')

In [8]:
df_onpaper_silent = regionprops1d(trace<0.5)

plt.figure()
plt.hist(df_onpaper_silent["Length"], bins=100)
plt.title('OnPaper Silent Duration')
plt.yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
from scipy.ndimage import binary_closing
OnPaperMask = binary_closing(trace>0.5, structure=np.ones(10))

df_onpaper = regionprops1d(OnPaperMask, raw.get_data(picks=['Type'])[0,:])
df_onpaper.drop(columns=['Mean', 'STD', 'Min', 'Max'], inplace=True)
df_onpaper.rename(columns={'Median': 'Type'}, inplace=True)

plt.figure()
plt.hist(df_onpaper.Onset.diff(), bins=100)
plt.title('Inter OnPaper')

fig, axx = plt.subplots(2, 5, figsize=(6,4), sharex=True, sharey=True)
for i, (ax, (g_name, g)) in enumerate(zip(axx.reshape(-1), df_onpaper.groupby('Type'))):
    plt.sca(ax)
    plt.hist(g.Length/raw.info['sfreq'], bins=np.arange(0, 4.1, 0.25))
#     plt.text(4, np.mean(plt.ylim()), f'{g_name-1:.0f}', fontsize=30, rotation=0, va='center')
    plt.title(i)
    if (i in np.arange(5,10)):
#         plt.xlabel('s')
        plt.xticks([0, 4], [0, '4s'])
    plt.xlim(0, 4)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    
plt.suptitle('Gesture Duration Histogram', y=1)

plt.tight_layout()

plt.savefig(os.path.join('figures', 'individual subject gesture duration.pdf'))
plt.savefig(os.path.join('figures', 'individual subject gesture duration.png'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Trial Events

In [10]:
events = mne.find_events(raw, stim_channel='Trial')

plt.figure()
plt.hist(np.diff(events[:,0])/raw.info['sfreq'], bins=100)
plt.title('Intertrial delay')

Trigger channel has a non-zero initial value of 1 (consider using initial_event=True to detect this event)
498 events found
Event IDs: [  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163
 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 2

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Intertrial delay')

In [11]:
raw.plot(scalings={'emg':1},
         events = events,
         duration=100);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Center events on OnPaper onset

In [12]:
events = mne.find_events(raw, stim_channel='Trial')
epochs = mne.Epochs(raw, events=events, tmin=0, tmax=5, preload=False, picks='all', baseline=None)
epochs.events[:,-1] = np.median(epochs.get_data(picks=['Type']), -1).reshape(-1).astype(int)

df_events = pd.DataFrame(epochs.events, columns=['ts', 'idx', 'code'])
print(df_events.shape)

s_shifts = pd.DataFrame(np.where(np.diff(epochs.get_data(picks=['OnPaper'])[:,0,:], axis=-1)>0.5)).T.groupby(0)[1].min()
s_shifts.name = 'shift'

df_events = pd.merge(df_events, s_shifts, left_index=True, right_index=True)

df_events.ts += df_events['shift']

on_paper_events = df_events.iloc[:, :3]

print(df_events.shape)

Trigger channel has a non-zero initial value of 1 (consider using initial_event=True to detect this event)
498 events found
Event IDs: [  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163
 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 2

### On paper epochs


In [13]:
epochs = mne.Epochs(raw, events=on_paper_events, 
                    tmin=-1, tmax=4,
                    preload=True, picks='all', baseline=None)

epochs.plot(scalings={'emg':1}, 
            picks='all', 
            events=epochs.events);

Not setting metadata
Not setting metadata
496 matching events found
No baseline correction applied
0 projection items activated
Loading data for 496 events and 51 original time points ...
0 bad epochs dropped


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Gesture Duration

In [14]:
offsets = pd.DataFrame(np.where(np.diff(epochs.get_data(picks=['OnPaper'])[:,0,:], axis=-1)<-0.5), 
                       index=('epoch', 'ts')).T.groupby('epoch').max()-10
plt.figure()
plt.hist(offsets, bins=100)
plt.title('Gesture Duration')
plt.axvline(4*epochs.info['sfreq'], color='r')

assert(offsets.ts.max()<40)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
epochs[offsets.ts>30].plot(scalings={'emg':1}, 
            picks='all', 
            events=epochs.events);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
fig, axx = plt.subplots(2, 5, sharex=True, sharey=True)

for i, (k, ax) in enumerate(zip(epochs.event_id.keys(), axx.reshape(-1))):
    plt.sca(ax)
    for X, Y, OnPaper in epochs[k].get_data(['X', 'Y', 'OnPaper']):
        mask = OnPaper>0.5
        plt.plot(X[mask], Y[mask], alpha=0.2, lw=1)
        plt.axis('image')
    
#     plt.xlim(xmin, xmax)
#     plt.ylim(ymin, ymax)
#     plt.title(k-1)
    plt.axis('off')
#     break
    
plt.tight_layout()

plt.savefig(os.path.join('figures', 'individual xy traces.pdf'))
plt.savefig(os.path.join('figures', 'individual xy traces.png'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Decomposition

In [17]:
epochs = mne.Epochs(raw, events=on_paper_events, 
                    tmin=0, tmax=4,
                    preload=True, picks='all', baseline=None)

X = epochs.get_data(picks='emg').reshape(len(epochs), -1)
y = epochs.events[:, -1]

Not setting metadata
Not setting metadata
496 matching events found
No baseline correction applied
0 projection items activated
Loading data for 496 events and 41 original time points ...
0 bad epochs dropped


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

pipe = Pipeline([('PCA', PCA(n_components=30)), 
                 ('TSNE', TSNE())])
tv = pipe.fit_transform(X)

In [19]:
plt.figure()
s = plt.scatter(*(tv.T), c=y-1, s=2, cmap=plt.get_cmap('jet', 10), vmin=-0.5, vmax=9.5)
c = plt.colorbar()
c.set_label('Gesture')
c.set_ticks(np.arange(10))
plt.box(False)

plt.title('Unsupervised EMG decomposition')
plt.xlabel('t1')
plt.ylabel('t2')

plt.savefig(os.path.join('figures', 'EMG decomposition.pdf'))
plt.savefig(os.path.join('figures', 'EMG decomposition.png'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### EMG figure

In [22]:
fig, axxx = plt.subplots(2, 10, sharex=True, sharey='row', gridspec_kw={'height_ratios':[5, 1]}, figsize=(6,6))
data = epochs[[1, 52, 101, 151, 201, 251, 301, 351, 401, 451]].get_data(picks=np.arange(10))
for i, (axx, d) in enumerate(zip(axxx.T, data)):
    ax = axx[0]
    plt.sca(ax)
    plt.plot(epochs.times, d[:8].T/3 - np.arange(8), color='k', lw=1)
    if i == 0:
        plt.yticks(-np.arange(8), epochs.ch_names[:8])
    plt.box(False)
    plt.title(i)
    
    ax = axx[1]
    plt.sca(ax)
    plt.plot(epochs.times, (d[8:]-d[8:,[0]]).T/150 - np.arange(2), color='C2', lw=1)
    if i == 0:
        plt.yticks(-np.arange(2), ['X', 'Y'])
        
    plt.xticks([0, 4], [0, '4s'])
    plt.box(False)
plt.suptitle('Individual Gesture Recordings', y=1)
plt.tight_layout(pad=.5)

plt.savefig(os.path.join('figures', 'individual emg.pdf'))
plt.savefig(os.path.join('figures', 'individual emg.png'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Classifier

In [23]:
from sklearn.model_selection import train_test_split

epochs = mne.Epochs(raw, events=on_paper_events, 
                    tmin=-1, tmax=4,
                    preload=True, picks='all', baseline=None)
epochs.events[:,-1] -=1

X = epochs.get_data(picks='emg')
y = epochs.events[:,-1].astype(int)-1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, stratify=y)
print('Train', X_train.shape, 'Test', X_test.shape)

Not setting metadata
Not setting metadata
496 matching events found
No baseline correction applied
0 projection items activated
Loading data for 496 events and 51 original time points ...
0 bad epochs dropped
Train (396, 8, 51) Test (100, 8, 51)


In [24]:
epochs[[1, 52, 101, 151, 201, 251, 301, 351, 401, 451]].plot(scalings={'emg':3, 
                                                                       'misc':150}, 
                                                             picks=np.arange(10), 
                                                             events=epochs.events);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Dataset

In [25]:
from sklearn.model_selection import train_test_split

epochs = mne.Epochs(raw, events=on_paper_events, 
                    tmin=-1, tmax=4,
                    preload=True, picks='all', baseline=None)

X = epochs.get_data(picks='emg')
y = epochs.events[:,-1].astype(int)-1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, stratify=y)
print('Train', X_train.shape, 'Test', X_test.shape)

Not setting metadata
Not setting metadata
496 matching events found
No baseline correction applied
0 projection items activated
Loading data for 496 events and 51 original time points ...
0 bad epochs dropped
Train (396, 8, 51) Test (100, 8, 51)


In [26]:
import torch
from torch.utils.data import Dataset, DataLoader

class GestureDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
        
    def __len__(self):
        return(len(self.X))
        
    def __getitem__(self, idx):
        sample = np.swapaxes(self.X[idx][..., np.newaxis, :, :], -1, -2)
        label  = self.y[idx]
        if self.transform:
            sample = self.transform(sample).double()
        return sample, label
        

In [27]:
from torch import functional as F
from torch import nn
sequence_length = len(epochs.times)
input_size  = 8
num_filters = 32
hidden_size = 32
num_layers  = 2
num_classes = 10
device='cuda'

class RNN(nn.Module):
    
    def __init__(self, input_size, num_filters, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_filters = num_filters
        
        self.cnn1 = nn.Conv2d(in_channels=1, 
                              out_channels=num_filters, 
                              kernel_size=(1, input_size), 
                              stride=1, 
                              padding=0)
        
        self.do1 = nn.Dropout2d(p=0.5)
        
        self.relu1 = nn.ReLU()
        
        self.lstm2 = nn.LSTM(num_filters, hidden_size, num_layers, batch_first=True, dropout=0.5)

#         self.relu2 = nn.ReLU()
        self.fc = nn.Linear(hidden_size, num_classes)
        pass
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Passing in the input and hidden state into the model and  obtaining outputs
        
        out = self.relu1(self.do1(self.cnn1(x)))
        out = torch.transpose(out, 1, 2)[..., 0]
        
        out, hidden = self.lstm2(out, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
#         out = self.relu2(out)
        
        #Reshaping the outputs such that it can be fit into the fully connected layer
        out = self.fc(out[:, -1, :]) # taking the last one in the sequence
        return out
       
        pass
pass
model = RNN(input_size, num_filters, hidden_size, num_layers, num_classes).to(device)
print(model)

RNN(
  (cnn1): Conv2d(1, 32, kernel_size=(1, 8), stride=(1, 1))
  (do1): Dropout2d(p=0.5, inplace=False)
  (relu1): ReLU()
  (lstm2): LSTM(32, 32, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=32, out_features=10, bias=True)
)


In [28]:
from torchinfo import summary
summary(model.cuda(), input_size=(100, 1, sequence_length, input_size))
        

Layer (type:depth-idx)                   Output Shape              Param #
RNN                                      --                        --
├─Conv2d: 1-1                            [100, 32, 51, 1]          288
├─Dropout2d: 1-2                         [100, 32, 51, 1]          --
├─ReLU: 1-3                              [100, 32, 51, 1]          --
├─LSTM: 1-4                              [100, 51, 32]             16,896
├─Linear: 1-5                            [100, 10]                 330
Total params: 17,514
Trainable params: 17,514
Non-trainable params: 0
Total mult-adds (M): 87.67
Input size (MB): 0.16
Forward/backward pass size (MB): 2.62
Params size (MB): 0.07
Estimated Total Size (MB): 2.85

## Single Run

In [29]:
from tqdm.autonotebook import tqdm

criterion = nn.CrossEntropyLoss()
softmax = nn.Softmax(1)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)   

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, stratify=y)
print('Train', X_train.shape, 'Test', X_test.shape)


train_loader = DataLoader(GestureDataset(X_train, y_train, transform=torch.from_numpy),
                          batch_size = len(y_train), 
                          shuffle = True)

test_loader = DataLoader(GestureDataset(X_test, y_test, transform=torch.from_numpy),
                          batch_size = len(y_test), 
                          shuffle = True)

num_epochs = 1000

plt.figure()

test_accs = []
for epoch in tqdm(range(num_epochs)):
    for i, (samples, labels) in enumerate(train_loader):
        samples = samples.to(device).float()
        labels = labels.to(device)

        # Forward pass
        outputs = model(samples)
        acc = (outputs.argmax(-1) == labels).cpu().numpy().mean()
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    plt.plot(epoch, loss.item(), '.', color='C0')

    if (epoch+1) % (num_epochs//100) == 0:
        with torch.no_grad():
            for i, (samples, labels) in enumerate(test_loader):
                samples = samples.to(device).float()
                labels = labels.to(device)

                # Forward pass
                outputs = model(samples)
                acc_val = (outputs.argmax(-1) == labels).cpu().numpy().mean()
                test_accs.append(acc_val)
                loss_val = criterion(outputs, labels)

        print ('Epoch [{}/{}], Step [{}], Loss: {:.2e}, Val: {:.2e}, Accuracy: {:.2f} vs {:.2f}' 
               .format(epoch + 1, num_epochs, i + 1, loss.item(), loss_val.item(), acc, acc_val))
        plt.plot(epoch, loss_val.item(), '.', color='C1')
        pass

    pass
pass

Train (396, 8, 51) Test (100, 8, 51)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch [10/1000], Step [1], Loss: 2.17e+00, Val: 2.22e+00, Accuracy: 0.18 vs 0.17
Epoch [20/1000], Step [1], Loss: 2.07e+00, Val: 2.05e+00, Accuracy: 0.19 vs 0.22
Epoch [30/1000], Step [1], Loss: 1.99e+00, Val: 1.92e+00, Accuracy: 0.24 vs 0.31
Epoch [40/1000], Step [1], Loss: 1.74e+00, Val: 1.85e+00, Accuracy: 0.35 vs 0.30
Epoch [50/1000], Step [1], Loss: 1.59e+00, Val: 1.53e+00, Accuracy: 0.37 vs 0.48
Epoch [60/1000], Step [1], Loss: 1.51e+00, Val: 1.45e+00, Accuracy: 0.39 vs 0.38
Epoch [70/1000], Step [1], Loss: 1.32e+00, Val: 1.35e+00, Accuracy: 0.51 vs 0.48
Epoch [80/1000], Step [1], Loss: 1.19e+00, Val: 1.25e+00, Accuracy: 0.53 vs 0.51
Epoch [90/1000], Step [1], Loss: 1.10e+00, Val: 1.28e+00, Accuracy: 0.55 vs 0.47
Epoch [100/1000], Step [1], Loss: 1.02e+00, Val: 1.07e+00, Accuracy: 0.60 vs 0.56
Epoch [110/1000], Step [1], Loss: 1.05e+00, Val: 1.12e+00, Accuracy: 0.60 vs 0.54
Epoch [120/1000], Step [1], Loss: 8.81e-01, Val: 9.71e-01, Accuracy: 0.65 vs 0.58
Epoch [130/1000], Step [1

In [30]:
from torchviz import make_dot

for i, (samples, labels) in enumerate(train_loader):
        samples = samples.to(device).float()
        labels = labels.to(device)
        break


dot = make_dot(model(samples), 
         params=dict(model.named_parameters()), 
         show_attrs=False, 
         show_saved=True, )

dot.render('/home/hp/myo/figures/rnn', format='pdf')

'/home/hp/myo/figures/rnn.pdf'

## KFold run

In [32]:
from tqdm.autonotebook import tqdm
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True)

metrics = {'acc_train': [], 'acc_val': [], 'loss_train': [], 'loss_val': []}

for train_index, test_index in kf.split(X, y):
    
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    print(train_index.shape, test_index.shape)
    
    train_loader = DataLoader(GestureDataset(X_train, y_train, transform=torch.from_numpy),
                              batch_size = len(y_train), 
                              shuffle = True)

    test_loader = DataLoader(GestureDataset(X_test, y_test, transform=torch.from_numpy),
                              batch_size = len(y_test), 
                              shuffle = True)

    num_epochs = 1000
    
    model = RNN(input_size, num_filters, hidden_size, num_layers, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    softmax = nn.Softmax(1)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)   

    plt.figure()

    loss_train_ = []
    loss_val_ = []
    acc_train_ = []
    acc_val_ = []
    
    for epoch in tqdm(range(num_epochs)):
        for i, (samples, labels) in enumerate(train_loader):
            samples = samples.to(device).float()
            labels = labels.to(device)

            # Forward pass
            outputs = model(samples)
            acc = (outputs.argmax(-1) == labels).cpu().numpy().mean()
            acc_train_.append(acc)
            
            loss = criterion(outputs, labels)
            loss_train_.append(loss.item())
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        plt.plot(epoch, loss.item(), '.', color='C0')

        if (epoch+1) % (num_epochs//100) == 0:
            with torch.no_grad():
                for i, (samples, labels) in enumerate(test_loader):
                    samples = samples.to(device).float()
                    labels = labels.to(device)

                    # Forward pass
                    outputs = model(samples)
                    acc_val = (outputs.argmax(-1) == labels).cpu().numpy().mean()
                    acc_val_.append(acc_val)
                    
                    loss_val = criterion(outputs, labels)
                    loss_val_.append(loss_val.item())

#             print ('Epoch [{}/{}], Step [{}], Loss: {:.2e}, Val: {:.2e}, Accuracy: {:.2f} vs {:.2f}' 
#                    .format(epoch + 1, num_epochs, i + 1, loss.item(), loss_val.item(), acc, acc_val))
            plt.plot(epoch, loss_val.item(), '.', color='C1')
            pass

        pass
    plt.title('Single Fold Training')
    
    metrics['acc_train'].append(acc_train_)
    metrics['acc_val'].append(acc_val_)
    metrics['loss_train'].append(loss_train_)
    metrics['loss_val'].append(loss_val_)
    pass


(396,) (100,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  0%|          | 0/1000 [00:00<?, ?it/s]

(397,) (99,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

  0%|          | 0/1000 [00:00<?, ?it/s]

(397,) (99,)


/tmp/ipykernel_70524/1221126587.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  0%|          | 0/1000 [00:00<?, ?it/s]

In [33]:
ep_train = np.arange(num_epochs)
ep_val = np.arange(0, num_epochs, 10)

fig, axx = plt.subplots(1,2, sharex=True, figsize=(6,3))

plt.sca(axx[0])
plt.plot(ep_train, np.array(metrics['loss_train']).T, lw=1, color='C0', alpha=0.25, label='__nolegend__')
plt.plot(ep_train, np.array(metrics['loss_train']).mean(0), lw=1, color='C0', alpha=1, label='Avg. Train loss (K=5)')

plt.plot(ep_val, np.array(metrics['loss_val']).T, lw=1, color='C1', alpha=0.25, label='__nolegend__')
plt.plot(ep_val, np.array(metrics['loss_val']).mean(0), lw=1, color='C1', alpha=1, label='Avg. CV loss (K=5)')
plt.legend(fontsize=10)
plt.grid(True)

plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.xlabel('Epoch')

plt.title('Cross-entropy loss')

plt.sca(axx[1])
plt.plot(ep_train, np.array(metrics['acc_train']).T, lw=1, color='C0', alpha=0.25, label='__nolegend__')
plt.plot(ep_train, np.array(metrics['acc_train']).mean(0), lw=1, color='C0', alpha=1, label='Avg. Train\nAccuracy (K=5)')

plt.plot(ep_val, np.array(metrics['acc_val']).T, lw=1, color='C1', alpha=0.25, label='__nolegend__')
plt.plot(ep_val, np.array(metrics['acc_val']).mean(0), lw=1, color='C1', alpha=1, label='Avg. CV\nAccuracy (K=5)')
plt.legend(fontsize=10)

plt.title('Accuracy')
plt.grid(True)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

plt.xlabel('Epoch')

plt.tight_layout()

plt.savefig(os.path.join('figures', 'individual subject training loss.pdf'))
plt.savefig(os.path.join('figures', 'individual subject training loss.png'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


# Intersubject metrics

## Utils

In [79]:
def mat2raw(mat):
    print('Creating raw')
    raw = mne.io.RawArray(np.concatenate([mat['export_EMG'], 
                mat['export_X'], 
                mat['export_Y'], 
                mat['export_OnPaper'], 
                mat['export_Type'], 
                mat['export_Trial']], axis=-1).T,
               mne.create_info(['emg%i'%i for i in range(8)] + ['X', 'Y', 'OnPaper', 'Type', 'Trial'], 
                               sfreq=1000,
                               ch_types=['emg']*8+['misc']*2+['stim']*3),
                         verbose=0)

    raw.apply_hilbert(picks='emg', envelope=True)
    raw.filter(0, 5, picks=['emg', 'misc']);
    raw.resample(10);
    return raw

In [98]:
def centerOnPaper(raw):
    print('Centering OnPaper events')
    events = mne.find_events(raw, stim_channel='Trial', verbose=0)
    epochs = mne.Epochs(raw, events=events, tmin=0, tmax=5, preload=False, picks='all', baseline=None)
    epochs.events[:,-1] = np.median(epochs.get_data(picks=['Type']), -1).reshape(-1).astype(int)

    df_events = pd.DataFrame(epochs.events, columns=['ts', 'idx', 'code'])
    print(df_events.shape)

    s_shifts = pd.DataFrame(np.where(np.diff(epochs.get_data(picks=['OnPaper'])[:,0,:], axis=-1)>0.5)).T.groupby(0)[1].min()
    s_shifts.name = 'shift'

    df_events = pd.merge(df_events, s_shifts, left_index=True, right_index=True)

    df_events.ts += df_events['shift']

    on_paper_events = df_events.iloc[:, :3]

    print(df_events.shape)
    return on_paper_events

## Dataset class

In [99]:
import torch
from torch.utils.data import Dataset, DataLoader

class GestureDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
        
    def __len__(self):
        return(len(self.X))
        
    def __getitem__(self, idx):
        sample = np.swapaxes(self.X[idx][..., np.newaxis, :, :], -1, -2)
        label  = self.y[idx]
        if self.transform:
            sample = self.transform(sample).double()
        return sample, label
        

## Model class

In [100]:
from torch import functional as F
from torch import nn
sequence_length = len(epochs.times)
input_size  = 8
num_filters = 32
hidden_size = 32
num_layers  = 2
num_classes = 10
device='cuda'

class RNN(nn.Module):
    
    def __init__(self, input_size, num_filters, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_filters = num_filters
        
        self.cnn1 = nn.Conv2d(in_channels=1, 
                              out_channels=num_filters, 
                              kernel_size=(1, input_size), 
                              stride=1, 
                              padding=0)
        
        self.do1 = nn.Dropout2d(p=0.5)
        
        self.relu1 = nn.ReLU()
        
        self.lstm2 = nn.LSTM(num_filters, hidden_size, num_layers, batch_first=True, dropout=0.5)

#         self.relu2 = nn.ReLU()
        self.fc = nn.Linear(hidden_size, num_classes)
        pass
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Passing in the input and hidden state into the model and  obtaining outputs
        
        out = self.relu1(self.do1(self.cnn1(x)))
        out = torch.transpose(out, 1, 2)[..., 0]
        
        out, hidden = self.lstm2(out, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
#         out = self.relu2(out)
        
        #Reshaping the outputs such that it can be fit into the fully connected layer
        out = self.fc(out[:, -1, :]) # taking the last one in the sequence
        return out
       
        pass
pass
model = RNN(input_size, num_filters, hidden_size, num_layers, num_classes).to(device)
print(model)

RNN(
  (cnn1): Conv2d(1, 32, kernel_size=(1, 8), stride=(1, 1))
  (do1): Dropout2d(p=0.5, inplace=False)
  (relu1): ReLU()
  (lstm2): LSTM(32, 32, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=32, out_features=10, bias=True)
)


## Run

In [103]:
from tqdm.autonotebook import tqdm
from sklearn.model_selection import StratifiedKFold

for idx_, row_ in tqdm(df_data.iterrows(), total=len(df_data)):
    matfile = row_.mat
    print('Loading', matfile)
    mat = loadmat(matfile)
    
    raw = mat2raw(mat)
    
    on_paper_events = centerOnPaper(raw)
    
    from sklearn.model_selection import train_test_split

    epochs = mne.Epochs(raw, events=on_paper_events, 
                        tmin=-1, tmax=4,
                        preload=True, picks='all', baseline=None, verbose=0)

    X = epochs.get_data(picks='emg')
    y = epochs.events[:,-1].astype(int)-1

    kf = StratifiedKFold(n_splits=5, shuffle=True)

    metrics = {'acc_train': [], 'acc_val': [], 'loss_train': [], 'loss_val': []}

    for train_index, test_index in kf.split(X, y):

        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]
        print(train_index.shape, test_index.shape)

        train_loader = DataLoader(GestureDataset(X_train, y_train, transform=torch.from_numpy),
                                  batch_size = len(y_train), 
                                  shuffle = True)

        test_loader = DataLoader(GestureDataset(X_test, y_test, transform=torch.from_numpy),
                                  batch_size = len(y_test), 
                                  shuffle = True)

        num_epochs = 1000

        model = RNN(input_size, num_filters, hidden_size, num_layers, num_classes).to(device)
        criterion = nn.CrossEntropyLoss()
        softmax = nn.Softmax(1)
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)   

        loss_train_ = []
        loss_val_ = []
        acc_train_ = []
        acc_val_ = []

        for epoch in tqdm(range(num_epochs)):
            for i, (samples, labels) in enumerate(train_loader):
                samples = samples.to(device).float()
                labels = labels.to(device)

                # Forward pass
                outputs = model(samples)
                acc = (outputs.argmax(-1) == labels).cpu().numpy().mean()
                acc_train_.append(acc)

                loss = criterion(outputs, labels)
                loss_train_.append(loss.item())
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            if (epoch+1) % (num_epochs//100) == 0:
                with torch.no_grad():
                    for i, (samples, labels) in enumerate(test_loader):
                        samples = samples.to(device).float()
                        labels = labels.to(device)

                        # Forward pass
                        outputs = model(samples)
                        acc_val = (outputs.argmax(-1) == labels).cpu().numpy().mean()
                        acc_val_.append(acc_val)

                        loss_val = criterion(outputs, labels)
                        loss_val_.append(loss_val.item())
                pass
            pass

        metrics['acc_train'].append(acc_train_)
        metrics['acc_val'].append(acc_val_)
        metrics['loss_train'].append(loss_train_)
        metrics['loss_val'].append(loss_val_)
        pass
    
    df_data.at[idx_, 'metrics'] = metrics   
        
#     break

  0%|          | 0/11 [00:00<?, ?it/s]

Loading export1_13Nov07.mat
Centering OnPaper events
Not setting metadata
Not setting metadata
498 matching events found
No baseline correction applied
0 projection items activated
Loading data for 498 events and 51 original time points ...
0 bad epochs dropped
(498, 3)
Loading data for 498 events and 51 original time points ...
(496, 4)
(396,) (100,)


  0%|          | 0/1000 [00:00<?, ?it/s]

(397,) (99,)


  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

(437,) (109,)


  0%|          | 0/1000 [00:00<?, ?it/s]

(437,) (109,)


  0%|          | 0/1000 [00:00<?, ?it/s]

In [114]:
np.array(metrics["acc_val"])[:,-10:].mean(-1)

array([0.95363636, 0.97798165, 0.9293578 , 0.96788991, 0.9559633 ])

In [ ]:
f'Accuracy {np.array(accs)[:, -10:].mean(-1).mean():.3f} ± {np.array(accs)[:, -10:].mean(-1).std():.3f}'

In [119]:
df_data.index.name='Index'

In [120]:
s_accs = df_data.metrics.apply(lambda m: f'{np.array(m["acc_val"])[:,-10:].mean(-1).mean():.3f}±{np.array(m["acc_val"])[:,-10:].mean(-1).std():.3f}')
s_accs.name = 'Acc. Mean±Std'
pd.merge(df_data, s_accs, left_index=True, right_index=True)[['mat', 'Acc. Mean±Std']]




,mat,Acc. Mean±Std
Index,,
0,export1_13Nov07.mat,0.934±0.022
1,export1_15Jan08.mat,0.972±0.012
2,export1_15Nov07.mat,0.900±0.015
3,export1_19Dec07.mat,0.809±0.032
4,export1_20Dec07.mat,0.895±0.015
5,export1_21Dec07.mat,0.793±0.042
6,export1_24Jan08.mat,0.924±0.018
7,export1_2Jan08.mat,0.949±0.007
8,export1_30Nov07.mat,0.702±0.030


In [123]:
df_data.metrics.apply(lambda m: np.array(m["acc_val"])[:,-10:].mean(-1).mean()).describe()

count    11.000000
mean      0.890751
std       0.086287
min       0.702054
25%       0.852127
50%       0.924413
75%       0.953036
max       0.972477
Name: metrics, dtype: float64